In [1]:
from openforcefield.topology import Molecule
from openforcefield.typing.engines.smirnoff import ForceField
from simtk.openmm.app import PDBFile 
from openforcefield.topology import Topology
from openforcefield.typing.engines.smirnoff import ForceField
from simtk.openmm import XmlSerializer
from pprint import pprint

Unable to load toolkit <openforcefield.utils.toolkits.OpenEyeToolkitWrapper object at 0x11dfa35c0>.


In [2]:
#mol = Molecule.from_smiles('CC')
lig = Molecule.from_file('ethane.sdf')

In [3]:
for idx, atom in enumerate(lig.atoms):
    atom.name = f'{atom.element.symbol}{idx}'
topology = Topology.from_molecules(lig)

In [4]:
forcefield = ForceField('test_forcefields/smirnoff99Frosst.offxml')

In [5]:
openmm_system = forcefield.create_openmm_system(topology)

In AmberToolsToolkitwrapper.computer_partial_charges_am1bcc: Molecule 'ethane' has more than one conformer, but this function will only generate charges for the first one.


In [6]:

pprint(XmlSerializer.serialize(openmm_system))

('<?xml version="1.0" ?>\n'
 '<System openmmVersion="7.3.1" type="System" version="1">\n'
 '\t<PeriodicBoxVectors>\n'
 '\t\t<A x="2" y="0" z="0"/>\n'
 '\t\t<B x="0" y="2" z="0"/>\n'
 '\t\t<C x="0" y="0" z="2"/>\n'
 '\t</PeriodicBoxVectors>\n'
 '\t<Particles>\n'
 '\t\t<Particle mass="12.01078"/>\n'
 '\t\t<Particle mass="12.01078"/>\n'
 '\t\t<Particle mass="1.007947"/>\n'
 '\t\t<Particle mass="1.007947"/>\n'
 '\t\t<Particle mass="1.007947"/>\n'
 '\t\t<Particle mass="1.007947"/>\n'
 '\t\t<Particle mass="1.007947"/>\n'
 '\t\t<Particle mass="1.007947"/>\n'
 '\t</Particles>\n'
 '\t<Constraints/>\n'
 '\t<Forces>\n'
 '\t\t<Force alpha="0" cutoff="1" dispersionCorrection="1" '
 'ewaldTolerance=".0005" forceGroup="0" ljAlpha="0" ljnx="0" ljny="0" ljnz="0" '
 'method="0" nx="0" ny="0" nz="0" recipForceGroup="-1" rfDielectric="78.3" '
 'switchingDistance="-1" type="NonbondedForce" useSwitchingFunction="0" '
 'version="3">\n'
 '\t\t\t<GlobalParameters/>\n'
 '\t\t\t<ParticleOffsets/>\n'
 '\t\t\t<Exc

In [7]:
import parmed as pmd
pmd_lig_struct = pmd.openmm.load_topology(topology.to_openmm(), openmm_system, lig.conformers[0])

In [8]:
pmd_lig_struct

<Structure 8 atoms; 1 residues; 7 bonds; parametrized>

In [9]:
pmd_lig_struct.save("ethane.prmtop", overwrite=True)
pmd_lig_struct.save("ethane.inpcrd", overwrite=True)

In [10]:
import BioSimSpace as BSS

/Users/toni_brain/miniconda3/envs/BSS/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::__1::vector<double, std::__1::allocator<double> > already registered; second conversion method ignored.
  return f(*args, **kwds)


In [11]:
ethane = BSS.IO.readMolecules(['ethane.prmtop','ethane.inpcrd'])

In [12]:
solv_ethane = BSS.Solvent.solvate("tip3p", molecule=ethane,
                                      box=3*[4*BSS.Units.Length.nanometer], work_dir='test')

In [13]:
BSS.IO.saveMolecules("solvated_ethane_smirnoff", solv_ethane, ["grotop", "gro87"])

['/Users/toni_brain/Projects/People/Jeff_Wagner/bssopenffhack/solvated_ethane_smirnoff.grotop',
 '/Users/toni_brain/Projects/People/Jeff_Wagner/bssopenffhack/solvated_ethane_smirnoff.gro87']

In [14]:
BSS.viewMolecules(["solvated_ethane_smirnoff.grotop","solvated_ethane_smirnoff.gro87"])

Reading molecules from '['solvated_ethane_smirnoff.grotop', 'solvated_ethane_smirnoff.gro87']'
Rendering the molecules...


_ColormakerRegistry()

ThemeManager()

In [19]:
BSS.IO.saveMolecules("ethane_from_parm", ethane, ['grotop', 'gro87'])

['/Users/toni_brain/Projects/People/Jeff_Wagner/bssopenffhack/ethane_from_parm.grotop',
 '/Users/toni_brain/Projects/People/Jeff_Wagner/bssopenffhack/ethane_from_parm.gro87']